In [1]:
import os
import geopandas as gpd

# os.environ["USE_PYGEOS"] = os.environ.get("USE_PYGEOS", "0")
local_crs = 32636
# path to data
example_data_path = "./data"

## Input data fetching
The input "blocks" data also should be obtained from the previous example result

In [2]:
from blocksnet.preprocessing import DataGetter, AggregateParameters

blocks = gpd.read_parquet(os.path.join(example_data_path, "blocks_cutter_result.parquet")).to_crs(local_crs)
getter = DataGetter(blocks=blocks)

## City blocks information aggregating
Here we aggregate information about buildings, greenings and parkings into the city blocks

In [3]:
buildings = gpd.read_parquet(os.path.join(example_data_path, "buildings.parquet"))
greenings = gpd.read_parquet(os.path.join(example_data_path, "greenings.parquet")).rename_geometry('geometry')
parkings = gpd.read_parquet(os.path.join(example_data_path, "parkings.parquet")).rename_geometry('geometry')

aggr_params = AggregateParameters(
  buildings=buildings,
  greenings=greenings,
  parkings=parkings
)

aggregated_blocks = getter.aggregate_blocks_info(params=aggr_params)
aggregated_blocks.head()

Restoring living area:   0%|          | 0/3748 [00:00<?, ?it/s]

Restoring living area squash:   0%|          | 0/3748 [00:00<?, ?it/s]

,geometry,landuse,block_id,is_living,current_population,floors,current_living_area,current_green_capacity,current_green_area,current_parking_capacity,current_industrial_area,area
0,"POLYGON ((344943.719 6647610.514, 344959.205 6...",buildings,0,False,0.0,NaN,0.0,0.0,0.0,0.0,0.0000,41.130907
1,"POLYGON ((344968.687 6647544.000, 344971.625 6...",buildings,1,False,0.0,NaN,0.0,0.0,0.0,0.0,0.0000,162.222101
2,"POLYGON ((344929.592 6647257.521, 344931.964 6...",buildings,2,False,0.0,0.0,0.0,0.0,0.0,0.0,42.6397,48.648312
3,"POLYGON ((344840.676 6647432.974, 344853.943 6...",buildings,3,False,0.0,NaN,0.0,0.0,0.0,0.0,0.0000,61.481533
4,"POLYGON ((344856.398 6647391.171, 344900.097 6...",buildings,4,False,0.0,NaN,0.0,0.0,0.0,0.0,0.0000,389.792373


## Accessibility matrix 
Accessibility matrix is generated with out intermodal ```GraphGenerator``` 

In [4]:
from blocksnet import GraphGenerator

city_geometry = gpd.read_parquet(os.path.join(example_data_path, 'city_geometry.parquet')).to_crs(local_crs)
transport_graph = GraphGenerator(city_geometry=city_geometry, local_crs=local_crs).get_graph('intermodal')
accessibility_matrix = getter.get_accessibility_matrix(transport_graph)

/home/vasilstar/masterplanning/blocksnet/preprocessing/accs_matrix_calculator.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.blocks.rename(columns={"index": "id"}, inplace=True)


In [5]:
accessibility_matrix.head()

id,0,1,2,3,4,5,6,7,8,9,...,1344,1345,1346,1347,1348,1349,1350,1351,1352,1353
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.1,8.2,10.0,10.0,8.2,5.5,3.9,3.9,13.2,...,30.1,25.5,24.5,24.5,25.3,27.3,28.8,28.8,28.4,25.0
1,0.1,0.0,8.1,9.9,9.9,8.1,5.6,4.0,4.0,13.3,...,30.2,25.6,24.6,24.6,25.4,27.4,28.7,28.7,28.5,25.1
2,8.2,8.1,0.0,2.0,2.0,0.0,13.6,11.6,11.6,9.2,...,30.7,26.1,25.1,25.1,25.9,28.8,27.7,27.7,30.9,27.5
3,10.0,9.9,2.0,0.0,0.0,2.0,12.0,13.4,13.4,7.2,...,31.3,26.7,25.7,25.7,26.5,30.4,29.5,29.5,28.9,25.5
4,10.0,9.9,2.0,0.0,0.0,2.0,12.0,13.4,13.4,7.2,...,31.3,26.7,25.7,25.7,26.5,30.4,29.5,29.5,28.9,25.5


Save result for the next example

In [6]:
aggregated_blocks.to_parquet(os.path.join(example_data_path, "data_getter_blocks.parquet"))
accessibility_matrix.to_pickle(os.path.join(example_data_path, "data_getter_matrix.pickle"))